### This notebook is experimental (fancier word for dirty) and unfinished.

In [ ]:
%matplotlib inline
import pandas as pd

## Read the data

In [ ]:
df1 = pd.read_csv('data/uniq_relation_data1.csv',
                 sep=';',
                 header=None)

In [ ]:
df1.head(2)

In [ ]:
df2 = pd.read_csv('data/uniq_relation_data2.csv',
                 sep=';',
                 header=None)

In [ ]:
df2.head(2)

In [ ]:
# Create list of unique pages / nodes and page2id dict
uniq = []
for i, r in df1.iterrows():
    uniq.append(list(r))
    
for i, r in df2.iterrows():
    uniq.append(list(r))
    
uniq_pages = sorted([i for i in list(set(itertools.chain.from_iterable(uniq))) if isinstance(i, str)])
print(f"{len(uniq_pages)} nodes in total\n{uniq_pages}")

In [ ]:
# Create list of init pages and related_1 pages
initial = list(df1[0])
related_1 = list(df2[0])

In [ ]:
page2id = {name: i for i, name in enumerate(uniq_pages)}
page2id

## Play with the content of posts

In [ ]:
## create a single column of pagenames
with open('data/data2-uniq.csv', 'r') as f:
    d = f.readlines()
    with open('data/pagenames2-uniq', 'w') as fw:
    for l in d:
        for page in l.split(';')[1:]:
            if page:
                fw.write(f"{page.strip()}\n")
                
## create a single column of links

In [ ]:
import json
import spacy

In [ ]:
with open('data-content-pagenames.json', 'r') as f:
    data = json.load(f)

In [ ]:
with open('data-content-pagenames1.json', 'r') as f:
    data1 = json.load(f)

In [ ]:
data.keys()

In [ ]:
for k, v in data.items():
    print(f"{k}: posts: {len(v)} words-rough: {len([ii for i in v for ii in i.split()])}")

In [ ]:
data['NatureClimateChange']

In [ ]:
# download and load english model
# !python3 -m spacy download en_core_web_sm
# nlp = spacy.load("en_core_web_sm")

# !python3 -m spacy download en_core_web_lg  # larger model contains much better word vectors
nlp = spacy.load("en_core_web_lg")

## Prepare word vectors / embeddings

In [ ]:
# post vector = average of token vectors (token = word) in the post
# each word/post vector has dimension 300
# non-usual words / out of vocabulary words get zero embeddings
# embeddings = {k: [nlp(post).vector for post in v] for k, v in data.items()}
embeddings = {k: nlp("\n\n".join([post for post in v])).vector for k, v in data.items()}
embeddings1 = {k: nlp("\n\n".join([post for post in v])).vector for k, v in data1.i1tems()}

In [ ]:
print(f"{len(data['cfact'])} {len(embeddings['cfact'])}")

## Tokenize and remove stop words

In [ ]:
# tokenized = {k: [token for post in v for token in nlp(post) if not token.is_stop] for k, v in data.items()}

In [ ]:
# tokenized['cfact'][0].lemma_

## Prepare data labels

In [ ]:
pos= ['greenpeace.international', 'NASAClimateChange', 'bloomberggreen', 'acespace', 'ClimateChangeIsReal',
      'climatereality', 'ClimateChangeNews', 'NatureClimateChange', 'ClimateChangeCauses', 'GlobalwarmingEva',
      'MostlyScience', 'EnvironmentandClimateChange', 'TheLeapOrg', 'NatureClimateChange', 'campaigncc',
      'climatedefenseproject', 'ClimateParents', ]

neg = ['ClimateChangeLies', 'cfact', 'iloveco2', 'SkepticalScience', 'ccdispatch', 'ClimateDepot', 'LordMonckton',
       '']

not_clima = ['fdrlst', 'JonahJGoldberg']

In [ ]:
for i in not_clima:
    if i in data1:
        print(i)

In [ ]:
for k in data.keys():
    if k in pos or k in neg or k in not_clima:
        print(k)

In [ ]:
sorted(data.keys())

In [ ]:
sorted(data1.keys())

## Split the data train/test 

## Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [ ]:
xy_clima_related = [(k, v, int(k in pos or k in neg)) for k, v in embeddings.items() if k in pos or k in neg or k in not_clima]

xy_clima_sentiment = [(k, v, int(k in pos)) for k, v in embeddings.items() if k in pos or k in neg]

In [ ]:
len(xy_clima)

In [ ]:
xy_clima_sentiment

In [ ]:
x_train, y_train = [], []
for d in xy_clima_sentiment:
    x_train.append(d[1])
    y_train.append(d[2])
    
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
y_train